In [119]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [120]:
plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']
plot_colours = ['#043ef9', '#eb5334', '#34eb37']

def plot_candles(df_plot, df_markers=None):
    fig = go.Figure()
    
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))
    if df_markers is not None:
        fig.add_trace(go.Candlestick(
            x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c,
            line=dict(width=1), opacity=1,
            increasing_fillcolor='#3480eb',
            decreasing_fillcolor='#3480eb',
            increasing_line_color='#3480eb',
            decreasing_line_color='#3480eb'
    ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True,
    )
    fig.show()

In [121]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [122]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [123]:
SMALL_BODY = 0.2
CLOSE_DISTANCE = 0.15
DOJI_BODY = 0.05
FULL_BODY = 0.95
CENTER_DISTANCE_HIGH = 0.55
CENTER_DISTANCE_LOW = 0.45
BIG_BODY = 0.6
ENGULFING_FACTOR = 1.1

def apply_top_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_h - row.mid_c
    else:
        return row.mid_h - row.mid_o
    
def apply_bottom_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_o - row.mid_l
    else:
        return row.mid_c - row.mid_l
    
def apply_hammer(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CLOSE_DISTANCE or row.DIST_BOTTOM_PERC < CLOSE_DISTANCE:
            return True
    return False

def apply_spinning_top(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CENTER_DISTANCE_HIGH or row.DIST_TOP_PERC < CENTER_DISTANCE_LOW:
            return True
    return False

def apply_engulfing(row):
    if row.PREV_DIRECTION != row.DIRECTION:
        if row.PREV_BODY_PERC > BIG_BODY and row.BODY_PERC > BIG_BODY:
            if row.BODY_RANGE > row.PREV_BODY_RANGE * ENGULFING_FACTOR:
                return True
    return False

def apply_stats(df):
    df['RANGE'] = df.mid_h - df.mid_l
    df['BODY_RANGE'] = abs(df.mid_c - df.mid_o)
    df['CENTER'] = (df.mid_h - df.mid_l) / 2 + df.mid_l
    df['BODY_PERC'] = df.BODY_RANGE / df.RANGE
    df['DIRECTION'] = df.mid_c - df.mid_o
    df['DIRECTION'] = df['DIRECTION'].apply(lambda x: 1 if x >= 0 else -1)
    df['DIST_TOP'] = df.apply(apply_top_end_distance, axis=1)
    df['DIST_BOTTOM'] = df.apply(apply_bottom_end_distance, axis=1)
    df['DIST_TOP_PERC'] = df.DIST_TOP / df.RANGE
    df['DIST_BOTTOM_PERC'] = df.DIST_BOTTOM / df.RANGE
    
    df['PREV_BODY_RANGE'] = df.BODY_RANGE.shift(1)
    df['PREV_BODY_PERC'] = df.BODY_PERC.shift(1)
    df['PREV_DIRECTION'] = df.DIRECTION.shift(1)
    
    df.dropna(inplace=True)
    
    return df

def apply_patterns(df):
    df['HAMMER'] = df.apply(apply_hammer, axis=1)
    df['SPINNING_TOP'] = df.apply(apply_spinning_top, axis=1)
    df['DOJI'] = df['BODY_PERC'].apply(lambda x: True if x < DOJI_BODY else False)
    df['MARUBOZU'] = df['BODY_PERC'].apply(lambda x: True if x > FULL_BODY else False)
    df['ENGULFING'] = df.apply(apply_engulfing, axis=1)
    
    return df

In [124]:
df = df_raw[cols].copy()
df = apply_stats(df)
df = apply_patterns(df)

In [125]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,RANGE,BODY_RANGE,CENTER,BODY_PERC,DIRECTION,...,DIST_TOP_PERC,DIST_BOTTOM_PERC,PREV_BODY_RANGE,PREV_BODY_PERC,PREV_DIRECTION,HAMMER,SPINNING_TOP,DOJI,MARUBOZU,ENGULFING
1,2020-01-02 02:00:00+00:00,108.671,108.758,108.658,108.742,0.100,0.071,108.708,0.710000,1,...,0.160000,0.130000,0.022,0.142857,1.0,False,False,False,False,False
2,2020-01-02 06:00:00+00:00,108.744,108.842,108.696,108.839,0.146,0.095,108.769,0.650685,1,...,0.020548,0.328767,0.071,0.710000,1.0,False,False,False,False,False
3,2020-01-02 10:00:00+00:00,108.836,108.866,108.658,108.694,0.208,0.142,108.762,0.682692,-1,...,0.144231,0.173077,0.095,0.650685,1.0,False,False,False,False,True
4,2020-01-02 14:00:00+00:00,108.697,108.748,108.212,108.529,0.536,0.168,108.480,0.313433,-1,...,0.095149,0.591418,0.142,0.682692,-1.0,False,False,False,False,False
5,2020-01-02 18:00:00+00:00,108.526,108.572,108.494,108.561,0.078,0.035,108.533,0.448718,1,...,0.141026,0.410256,0.168,0.313433,-1.0,False,False,False,False,False


In [128]:
df_plot = df.iloc[700:800]
df_markers = df_plot[(df_plot.ENGULFING==True)]

In [129]:
plot_candles(df_plot, df_markers)